## Importing required modules

In [1]:
# importing required modules
import math
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold

## Loading data

In [2]:
train_data = pd.read_csv("../input/30-days-of-ml/train.csv")
test_data = pd.read_csv("../input/30-days-of-ml/test.csv")

#### Making K-fold column in training data

In [3]:
train_data["kfold"] = -1

## K-Fold model, total 10 splits

In [4]:
Kf_model = KFold(n_splits=10, random_state=1, shuffle=True)
for fold, (train_index, valid_index) in enumerate(Kf_model.split(X=train_data)):
    train_data.loc[valid_index, "kfold"] = fold

In [5]:
print(train_data.shape)
train_data.head()

(300000, 27)


,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,...,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,9
1,2,B,B,A,A,B,D,A,F,A,...,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,1
2,3,A,A,A,C,B,D,A,D,A,...,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,8
3,4,B,B,A,C,B,D,A,E,C,...,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,2
4,6,A,A,A,C,B,D,A,E,A,...,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,1


### Categorical data column and Numerical data column

In [6]:
# Prediction target and features
cat_cols = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
num_cols = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13']
test_data_2 = test_data[cat_cols + num_cols]

## Getting final predictions & RMSE on each fold

In [7]:
final_predictions = []
rmse_scores = []

# As total folds are 10, getting predictions and rmse at every fold
for fold in range(10):
    # Splitting the training and validating data according to fold and kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    
    X_test = test_data_2.copy()
    
    # setting the target
    y_train = X_train.target
    y_valid = X_valid.target
    
    # setting the training data
    X_train = X_train[cat_cols + num_cols]
    X_valid = X_valid[cat_cols + num_cols]
    
    # One Hot Encoding on categorical data
    OH_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
    
    # fitting and transforming the training and test data
    OH_encoder.fit(X_train[cat_cols])
    OH_X_train = OH_encoder.transform(X_train[cat_cols])
    OH_X_valid = OH_encoder.transform(X_valid[cat_cols])
    OH_X_test = OH_encoder.transform(X_test[cat_cols])
    
    # Naming the one hot encoded columns
    OH_X_train = pd.DataFrame(OH_X_train, columns=[f"ohe_{i}" for i in range(OH_X_train.shape[1])])
    OH_X_valid = pd.DataFrame(OH_X_valid, columns=[f"ohe_{i}" for i in range(OH_X_valid.shape[1])])
    OH_X_test = pd.DataFrame(OH_X_test, columns=[f"ohe_{i}" for i in range(OH_X_test.shape[1])])
    
    # Adding one hot encoded columns to main data (training, validating, test)
    X_train = pd.concat([X_train, OH_X_train], axis=1)
    X_valid = pd.concat([X_valid, OH_X_valid], axis=1)
    X_test = pd.concat([X_test, OH_X_test], axis=1)
    
    # Dropping the categorical columns, as their one hot encoded columns are added to main data
    X_train = X_train.drop(cat_cols, axis=1)
    X_valid = X_valid.drop(cat_cols, axis=1)
    X_test = X_test.drop(cat_cols, axis=1)
    
    # Making XGBoost Regressor model
    model = XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor="gpu_predictor")
    model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)
    
    # pridicting the target on training and test data
    train_preds = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    
    # RMSE at each prediction
    rmse = math.sqrt(mean_squared_error(y_valid, train_preds))
    rmse_scores.append(rmse)
    
    print(f"Fold: {fold}, RMSE: {rmse}, Predictions: {test_preds}")


Fold: 0, RMSE: 0.7193209740496443, Predictions: [8.079874 8.315076 8.411264 ... 8.357581 8.165665 8.014028]
Fold: 1, RMSE: 0.7193958876999159, Predictions: [8.060634  8.351181  8.362048  ... 8.394792  8.183628  7.9629917]
Fold: 2, RMSE: 0.7238844909277737, Predictions: [8.094143 8.358214 8.344183 ... 8.375742 8.053192 8.058901]
Fold: 3, RMSE: 0.7305995262635391, Predictions: [8.024538 8.274868 8.370956 ... 8.391476 7.968307 7.974936]
Fold: 4, RMSE: 0.7238212757348244, Predictions: [8.077593  8.280942  8.3715515 ... 8.375725  8.064892  8.055317 ]
Fold: 5, RMSE: 0.7219545504726438, Predictions: [8.029388 8.31109  8.357372 ... 8.385788 8.020295 8.009853]
Fold: 6, RMSE: 0.7204353124915598, Predictions: [7.9479933 8.317362  8.353496  ... 8.340463  8.051353  8.017334 ]
Fold: 7, RMSE: 0.7237808508396867, Predictions: [8.074957  8.325325  8.4148655 ... 8.355898  8.06382   7.939054 ]
Fold: 8, RMSE: 0.7193054476112654, Predictions: [8.117125 8.298847 8.364904 ... 8.315409 8.101424 8.023036]
Fold

### Getting the final predicitons of target from `final_predictions` arrays of arrays

In [8]:
final_predictions =  np.mean(np.column_stack(final_predictions), axis=1)
final_predictions

array([8.0481205, 8.317375 , 8.368177 , ..., 8.36452  , 8.0796795,
       8.005109 ], dtype=float32)

## Submitting the final predictions

In [9]:
# Submitting predicted target to `submission.csv`
output = pd.DataFrame({'Id': test_data.id,'target': final_predictions})
output.to_csv('submission.csv', index=False)